In [1]:
import pandas as pd
import numpy as np
import re
import difflib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Check Data

In [3]:
drive = "https://drive.google.com/uc?id="
df_sinta_label = pd.read_json("https://raw.githubusercontent.com/glorianatasyaaa/oplib-advanced-search-scraper/main/preprocessSinta/clean-name_sinta-label.json")
df_sinta_nolabel = pd.read_json("https://raw.githubusercontent.com/glorianatasyaaa/oplib-advanced-search-scraper/main/preprocessSinta/df_sinta_nolabel.json")
df_dosen = pd.read_excel(drive + "1IHgZr5povv7tgblFAcWDv0ImZybocAs0")

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
df_sinta_label.shape

(17, 20)

In [6]:
df_sinta_label.head()

,Judul,Penulis,Abstrak,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15,SDG16,SDG17
0,Solid oxide fuel cell energy system with absorption-ejection refrigeration optimized using a neural network with multiple objectives,"Hai Taoa, Farhan A. Alenizi, Adil Husseine Mohammed, Vishalf Goyal, Riyam K. Marjan, Kameliah Quzwain, Ahmed Sayedi Mohammed Metwally",teliti fokus model bangkit listrik sel bahan bakar oksida padat kombinasi siklus dingin serap pertama banding buat siklus dingin chiller serap chiller serap hubung siklus superior sel bahan bakar oksida padat siklus tambah kemudian siklus sel bahan bakar oksida padat bakar produk output unit pulih panas kombinasi siklus dingin produksi air tawar model lanjut analisis sensitivitas saji ajar efek parameter desain fungsi objektif sederhana benar hasil optimasi dasar algoritma genetika laku optimasi metode ajar mesin guna kurang waktu biaya komputasi optimalisasi siklus tunjuk efisiensi eksergi tingkat hingga tingkat biaya seluruh dolar per jam,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Changes in Indonesian private universities educational practices in the post COVID-19 environment,"Grisnaa Anggadwita, Nurul Indarti, Vanessa Ratten",didik salah satu sektor paling penting pengaruh covid turun sosial syarat studi rumah akibat lembaga didik ubah proses ajar hal bagaimana kelas sampai ada sulit pandemi covid juga hasil peluang unik terap transformasi digital sistem didik wujud ubah positif akibat didik siswa harus dapat cepat adaptasi ubah sistem ajar online lembaga didik investasi baru infrastruktur kembang platform ajar online fasilitas proses ajar ajar guna lensa model ubah lewin teliti tuju eksplorasi efek pandemi covid praktik didik perspektif didik tentu utama perilaku kesiap universitas studi guna metode kualitatif laku wawancara dalam semi struktur tujuh belas informan kunci rupa didik universitas swasta indonesia studi temu bagi besar didik alami masalah lama praktik ajar online mereka sadar ubah praktik didik akan lanjut era pasca covid juga identifikasi tentu utama kesiap perilaku urus ubah inisiatif mampu adaptasi libat inovasi praktik didik bantu paham proses digitalisasi sebab pandemi covid studi milik implikasi praktis akademik beri rekomendasi buat bijak didik praktisi teliti sistem didik pasca covid bagaimana tangan krisis masa depan,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"Website Quality Analysis, Trust and Loyalty of Indonesian Railway Service","Mikael Julian Irsa, Mahir Pradana",studi maksud temu analisis situs web kualitas percaya loyalitas konsumen giat transaksi online cepat tingkat jumlah klien online indonesia tumbuh tahun tahun hal jumlah toko online jual media bas internet beda peluang toko online sedang kembang saing tingkat langgan percaya loyal milik bisnis online paham penuh ingin langgan studi guna metodologi teliti kuantitatif ukur sampel responden analis guna analisis jalur edisi tiga program smartpls data oleh pilih lebih data tidak perlu hapus saring proses lalu kumpul data tahap perlu temu analisis sangat maksimal tentu hasil akhir hasil teliti tunjuk semua faktor milik pengaruh positif substansial hasil sangat bagus dapat terima,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,AI in Indian Libraries: Prospects and Perceptions from Library Professionals,"Subaveerapandiyan A., Alfian Akbar Gozali",studi eksplorasi perspektif profesional pustaka india guna cerdas buat ai pustaka tuju paham tahu sadar pandang ai tantang peluang teliti adopsi dekat kuantitatif guna survei tutup kumpul data profesional pustaka lembaga akademik seluruh india survei cakup bagai aspek masuk informasi sosial demografis tahu ai perspektif ai pustaka timbang etis adopsi alat layan ai temu tunjuk para profesional pustaka india umum sadar ai potensi manfaat pustaka percaya ai tingkat giat pustaka tingkat aksesibilitas dukung ambil putus khawatir tentang ai ganti cerdas manusia dalam pustaka faktor privasi guna dana ahli staf selaras tuju lembaga sangat penting terap apl

In [7]:
df_sinta_label['Penulis'][0]

'Hai Taoa, Farhan A. Alenizi, Adil Husseine Mohammed, Vishalf Goyal, Riyam K. Marjan, Kameliah Quzwain, Ahmed Sayedi Mohammed Metwally'

# Methods and Functions

In [8]:
def ratio(first, second):
  return difflib.SequenceMatcher(None, first, second).ratio()

def is_similar(first, second, threshold):
  return ratio(first, second) > threshold and ((ratio(first.split()[0], second.split()[0]) == 1) or ((len(" ".join(first.split()[1:])) > 2) and ratio(" ".join(first.split()[1:]), " ".join(second.split()[1:])) > threshold))

In [9]:
def name_corrector(names_list, change_df) :
    result = []
    for name in names_list:
        max_ratio = 0.
        best_match = name
        for _, row in change_df.iterrows():
            if is_similar(name, row['NAMA LENGKAP'], 0.9):
                current_ratio = ratio(name, row['NAMA LENGKAP'])
                if current_ratio > max_ratio:
                    max_ratio = current_ratio
                    best_match = row['NAMA LENGKAP']
                print(name)
                print(best_match, '\n')
        result.append(best_match)
    return result

In [10]:
def process_names(df, column_label, change_df):
    df['Nama_Awal'] = df[column_label].apply(lambda x: x.split(','))
    df['Nama_Betul'] = df['Nama_Awal'].apply(lambda x: name_corrector(x, change_df))
    df['Penulis'] = df['Nama_Betul'].apply(lambda x: ', '.join(x))
    df.drop(['Nama_Awal', 'Nama_Betul'], axis=1, inplace=True)
    return df

In [11]:
def clean_names(input_string):
    if pd.isna(input_string):
        return input_string

    # Remove the part starting from "Send mail to" until before semicolon or end of string
    input_string = re.sub(r'Send mail to[^;]*', '', input_string)

    # Extract all parts after semicolons and commas
    parts = re.split(r';', input_string)

    cleaned_names = []
    seen_names = set()

    for part in parts:
        part = part.strip()

        # Arrange names with more than one comma to ensure proper separation
        name_parts = part.split(',')
        arranged_parts = []
        temp_part = ''
        for i in range(len(name_parts)):
            temp_part = temp_part + name_parts[i].strip()
            if i % 2 == 1 or i == len(name_parts) - 1:
                arranged_parts.append(temp_part)
                temp_part = ''
            else:
                temp_part = temp_part + ', '

        # Process each arranged part
        for name in arranged_parts:
            # Remove trailing alphabetical sequence
            name = re.sub(r'([a-z])$', '', name).strip()

            # Remove trailing comma
            name = re.sub(r',\s*$', '', name).strip()

            # Swap last name and first name
            split_name = name.split(',')
            if len(split_name) == 2:
                last_name = split_name[0].strip()
                first_name = split_name[1].strip()
                formatted_name = f"{first_name} {last_name}"
            else:
                formatted_name = name

            # Add to cleaned names if not a duplicate
            if formatted_name not in seen_names:
                cleaned_names.append(formatted_name)
                seen_names.add(formatted_name)

    return ', '.join(cleaned_names)

In [12]:
def remove_invalid_commas(input_string):
    # Remove sequences of commas and whitespace that are not likely to be part of names
    cleaned_string = re.sub(r',\s*,', ',', input_string)
    # cleaned_string = re.sub(r'\s{3,}', ',', cleaned_string)  # Replace sequences of 3 or more spaces with a single space
    return cleaned_string

In [13]:
def remove_single_characters(input_string):
    # Use regex to remove single characters between commas
    cleaned_string = re.sub(r',\s*[a-zA-Z]\s*,', ',', input_string)
    return cleaned_string

In [14]:
def remove_excessive_whitespace(input_string):
    # Use regex to replace multiple spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', input_string)
    return cleaned_string.strip()

# Cleaning

In [15]:
df_dosen['NAMA LENGKAP'] = df_dosen['NAMA LENGKAP'].str.title()
df_dosen.head()

,NO,NIP,FRONTTITLE,NAMA LENGKAP,BACKTITLE,JENIS KELAMIN,KODE DOSEN,NIDN,JAFUNG,LOKASI KERJA,JABATAN STRUKTURAL,STATUS PEGAWAI,EMAIL,NO HP,LOKASI KERJA SOTK
0,1,21930012-3,Dr.,A. Hasan Al Husain,"S.IP., M.I.Kom.",PRIA,HSU,2.110059e+09,AA 3,PROGRAM STUDI S1 ILMU KOMUNIKASI (FKB),NaN,DOSEN PROFESIONAL FULL TIME,a.hasanalhusain@gmail.com,085159951544,FAKULTAS KOMUNIKASI DAN BISNIS
1,2,23660002-3,Dr. Ir.,A. Mukti Soma,M.M.,PRIA,DUL,8.917310e+09,L 3,PROGRAM STUDI S2 MANAJEMEN (FEB),NaN,DOSEN PROFESIONAL FULL TIME,NaN,08112245599,FAKULTAS EKONOMI DAN BISNIS
2,3,01730036-1,NaN,Aang Saeful Anwar,NaN,PRIA,NaN,NaN,Muda 2,"URUSAN MECHANICAL, ELECTRICAL DAN SIPIL",NaN,TPA PEGAWAI TETAP,aangsa@telkomuniversity.ac.id,081220391628 # 08982392938,DIREKTORAT ASET DAN SUSTAINABILITY
3,4,23990046-3,NaN,Aaz Muhammad Hafidz Azis,"S.T., M.T.",PRIA,ZHH,NaN,NJFA 2,PROGRAM STUDI S1 REKAYASA PERANGKAT LUNAK (FIF),NaN,DOSEN PROFESIONAL FULL TIME,Aazmhafidz09@gmail.com,085723120000,FAKULTAS INFORMATIKA
4,5,19930025-1,NaN,Abd. Rahman N,S.ST.,PRIA,NaN,NaN,Madya 1,URUSAN ENDOWMENT,KEPALA URUSAN ENDOWMENT,TPA PEGAWAI TETAP,abdrahman@telkomuniversity.ac.id # for.rahman01@gmail.com,081214757333 - kartu halo # 082116222212 - non halo,"DIREKTORAT PENGEMBANGAN KARIR, ALUMNI DAN ENDOWMENT"


In [17]:
df_sinta_label = process_names(df_sinta_label, 'Penulis', df_dosen)
df_sinta_label['Penulis'] = df_sinta_label['Penulis'].apply(remove_single_characters)
df_sinta_label['Penulis'] = df_sinta_label['Penulis'].apply(remove_invalid_commas)
df_sinta_label['Penulis'] = df_sinta_label['Penulis'].apply(remove_excessive_whitespace)

Grisnaa Anggadwita
Grisna Anggadwita 

 Mahir Pradana
Mahir Pradana 

 Alfian Akbar Gozali
Alfian Akbar Gozali 

 Dodie Tricahyono
Dodie Tricahyono 

 Dian Indiyati
Dian Indiyati 

 Ridha Muldina Negara
Ridha Muldina Negara 

 Sofia Naninga Hertiana
Sofia Naning Hertiana 

 Rohmat Tulloh
Rohmat Tulloh 

Iwan Iwut Tritoasmoro
Iwan Iwut Tritoasmoro 

Grisnaa Anggadwita
Grisna Anggadwita 

 Mahir Pradana
Mahir Pradana 

 Indrawati
Indrawati 

 Rezzy Eko Caraka
Rezzy Eko Caraka 

 Ari Yanuar Ridwan
Ari Yanuar Ridwan 

 Ari Yanuar Ridwan
Ari Yanuar Ridwan 

 Umar Yunan Kurnia Septo Hediyanto
Umar Yunan Kurnia Septo Hediyanto 



In [18]:
df_sinta_label.to_json('clean-name_sinta-label.json', orient='records')

In [19]:
df_sinta_nolabel = process_names(df_sinta_nolabel, 'Penulis', df_dosen)
df_sinta_nolabel['Penulis'] = df_sinta_nolabel['Penulis'].apply(remove_single_characters)
df_sinta_nolabel['Penulis'] = df_sinta_nolabel['Penulis'].apply(remove_invalid_commas)
df_sinta_nolabel['Penulis'] = df_sinta_nolabel['Penulis'].apply(remove_excessive_whitespace)

Streaming output truncated to the last 5000 lines.
Irfan Darmawan 

Satria Mandala
Satria Mandala 

Agus Pratondo
Agus Pratondo 

 Astri Novianty
Astri Novianty 

 Nungki Selviandro
Nungki Selviandro 

 Kusuma Ayu Laksitowening
Kusuma Ayu Laksitowening 

 Veronikha Effendy
Veronikha Effendy 

 Monterico Adrian
Monterico Adrian 

 Ratna Mayasari
Ratna Mayasari 

 Arif Indra Irawan
Arif Indra Irawan 

 Kris Sujatmoko
Kris Sujatmoko 

 Brian Pamukti
Brian Pamukti 

 Rio Nurtantyana
Rio Nurtantyana 

Purba Daru Kusuma
Purba Daru Kusuma 

 Dimas Adiputra
Dimas Adiputra 

Nova Indah Saragih
Nova Indah Saragih 

 Suprayogi
Suprayogi 

Agus D. Prasetyo
Agus Dwi Prasetyo 

Said Al Faraby
Said Al Faraby 

 Adiwijaya
Adiwijaya 

 Ade Romadhony
Ade Romadhony 

 Mamat Rokhmat
Mamat Rokhmat 

 Edy Wibowo
Edy Wibowo 

Yus Natali
Yus Natali 

 Leanna Vidya Yovita
Leanna Vidya Yovita 

 Tody Ariefianto Wibowo
Tody Ariefianto Wibowo 

Rezzy Eko Caraka
Rezzy Eko Caraka 

 Marastika Wicaksono Aji Bawono
M

In [20]:
df_sinta_nolabel.to_json('clean-name_sinta-nolabel.json', orient='records')